# Import Library

In [6]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

# Preprocessing

In [77]:
import pandas as pd

# Membaca dataset
df = pd.read_csv('merged_dataset_untuk_jurnal.csv')

print(df.head(3))


         mean_1  standard_deviation_1  median_1  min_value_1  max_value_1  \
0 -10087.594937           1095.953766  -10445.0       -14409        -7321   
1  -8315.313791           1203.245256   -8298.0       -14163        -2830   
2 -11952.875455           1373.377233  -11634.5       -18896        -7042   

   range_1  interquartile_range_1     q1_1      q3_1  kurtosis_1  skewness_1  \
0     7088                1950.00 -10895.0  -8945.00   -0.844598    0.024230   
1    11333                 771.00  -8675.0  -7904.00    6.035557   -0.319061   
2    11854                1283.25 -12461.0 -11177.75    3.624293   -1.143653   

  station_1 channel_1 class_1     mean_2  standard_deviation_2  median_2  \
0     MELAB       HHN  BEBENG -21.166556           3986.992718     -56.0   
1     MELAB       HHZ  BEBENG  -9.502998           1738.396078     -36.0   
2     MELAB       HHE  BEBENG  26.772818           3441.059767      14.0   

   min_value_2  max_value_2  range_2  interquartile_range_2    q1

In [62]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from imblearn.over_sampling import SMOTE

# Assuming merged_df is your DataFrame
# Separate features and labels
X = df.drop(['class_1', 'class_2', 'station_1', 'station_2', 'channel_1', 'channel_2'], axis=1)
y = df['class_1']

# Label encoding for the target label
label_encoder = LabelEncoder()
y_encoded = label_encoder.fit_transform(y)

# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X, y_encoded, test_size=0.4, stratify=y_encoded, random_state=42)

# Apply SMOTE to the training data
smote = SMOTE(random_state=42)
X_train_smote, y_train_smote = smote.fit_resample(X_train, y_train)

In [63]:
print("========================SEBELUM SMOTE========================== \n")

# Mendapatkan jumlah data untuk setiap kelas dalam y_train dan y_test
train_class_counts = pd.Series(y_train).value_counts()
test_class_counts = pd.Series(y_test).value_counts()

# Menampilkan jumlah data untuk setiap kelas
print("Data Latih:")
for class_label, count in train_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")

print("\nData Uji:")
for class_label, count in test_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")
    
print("========================SESUDAH SMOTE========================== \n")
# Mendapatkan jumlah data untuk setiap kelas dalam y_train dan y_test
train_class_counts = pd.Series(y_train_smote).value_counts()
test_class_counts = pd.Series(y_test).value_counts()

print("Data Latih:")
for class_label, count in train_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")

print("\nData Uji:")
for class_label, count in test_class_counts.items():
    print(f"{label_encoder.inverse_transform([class_label])[0]} = {count} data")  
    


========================SEBELUM SMOTE========================== 

Data Latih:
BEBENG = 653 data
BOYONG = 31 data
GENDOL = 26 data

Data Uji:
BEBENG = 436 data
BOYONG = 20 data
GENDOL = 18 data
========================SESUDAH SMOTE========================== 

Data Latih:
BEBENG = 653 data
GENDOL = 653 data
BOYONG = 653 data

Data Uji:
BEBENG = 436 data
BOYONG = 20 data
GENDOL = 18 data


In [64]:
class_names = df['class_1'].unique()

In [67]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN
knn_model = KNeighborsClassifier()

# Tentukan rentang nilai K yang ingin dieksplorasi
param_dist = {
    'n_neighbors': range(1, 28),
    'p' : [2]
}
  # Contoh: kita ingin mencoba nilai K dari 1 hingga 20

# Inisialisasi objek RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=knn_model, param_distributions=param_dist, n_iter=100, cv=10, scoring='accuracy', random_state=42)

# Latih model menggunakan Random Search
random_search.fit(X_train_smote, y_train_smote)

# Tampilkan hasil pencarian parameter
print("Best parameter (K):", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Evaluasi model terbaik pada data uji
best_knn_model = random_search.best_estimator_
y_pred = best_knn_model.predict(X_test)

# Tampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=class_names))


d:\Python 3.11.8\Lib\site-packages\sklearn\model_selection\_search.py:318: UserWarning: The total space of parameters 27 is smaller than n_iter=100. Running 27 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(


Best parameter (K): {'p': 2, 'n_neighbors': 1}
Best cross-validation score: 0.9928519099947671
              precision    recall  f1-score   support

      BEBENG       0.99      0.98      0.98       436
      BOYONG       0.64      0.70      0.67        20
      GENDOL       1.00      1.00      1.00        18

    accuracy                           0.97       474
   macro avg       0.87      0.89      0.88       474
weighted avg       0.97      0.97      0.97       474



In [7]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN
knn_model = KNeighborsClassifier()

# Tentukan rentang nilai K yang ingin dieksplorasi
param_dist = {
    'n_neighbors': range(1, 28),
    'p' : [2]
}
  # Contoh: kita ingin mencoba nilai K dari 1 hingga 20

# Inisialisasi objek RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=knn_model, param_distributions=param_dist, n_iter=100, cv=5, scoring='accuracy', random_state=42)

# Latih model menggunakan Random Search
random_search.fit(X_train_smote, y_train_smote)

# Tampilkan hasil pencarian parameter
print("Best parameter (K):", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Evaluasi model terbaik pada data uji
best_knn_model = random_search.best_estimator_
y_pred = best_knn_model.predict(X_test)

# Tampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=class_names))


Best parameter (K): {'p': 2, 'n_neighbors': 1, 'leaf_size': 51}
Best cross-validation score: 0.9880062344139651
              precision    recall  f1-score   support

      BEBENG       0.99      0.97      0.98        74
      BOYONG       0.60      0.75      0.67         4
      GENDOL       1.00      1.00      1.00         4

    accuracy                           0.96        82
   macro avg       0.86      0.91      0.88        82
weighted avg       0.97      0.96      0.97        82



In [8]:
from sklearn.model_selection import RandomizedSearchCV
from sklearn.neighbors import KNeighborsClassifier

# Inisialisasi model KNN
knn_model = KNeighborsClassifier()

# Tentukan rentang nilai K yang ingin dieksplorasi
param_dist = {
    'n_neighbors': range(1, 28),
    'p' : [2]
}
  # Contoh: kita ingin mencoba nilai K dari 1 hingga 20

# Inisialisasi objek RandomizedSearchCV
random_search = RandomizedSearchCV(estimator=knn_model, param_distributions=param_dist, n_iter=100, cv=10, scoring='accuracy', random_state=42)

# Latih model menggunakan Random Search
random_search.fit(X_train_smote, y_train_smote)

# Tampilkan hasil pencarian parameter
print("Best parameter (K):", random_search.best_params_)
print("Best cross-validation score:", random_search.best_score_)

# Evaluasi model terbaik pada data uji
best_knn_model = random_search.best_estimator_
y_pred = best_knn_model.predict(X_test)

# Tampilkan laporan klasifikasi
print(classification_report(y_test, y_pred, target_names=class_names))


Best parameter (K): {'p': 2, 'n_neighbors': 1, 'leaf_size': 51}
Best cross-validation score: 0.9885124378109452
              precision    recall  f1-score   support

      BEBENG       0.99      0.97      0.98        74
      BOYONG       0.60      0.75      0.67         4
      GENDOL       1.00      1.00      1.00         4

    accuracy                           0.96        82
   macro avg       0.86      0.91      0.88        82
weighted avg       0.97      0.96      0.97        82



In [9]:
# Simpan model yang telah dilatih
import joblib
model_filename = 'model_KNN_9010.pkl'
joblib.dump(best_knn_model, model_filename)
print(f"Model saved to {model_filename}")

Model saved to model_KNN_9010.pkl
